In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
sns.set_style("whitegrid")

First we load the data (data comes from cleaning the full dataset; see Clean_Full_Dataset.ipynb for details), drop any rows with NAs, and convert the 'CheckoutDate' column to a DateTime datatype. We also sort by CheckoutDate and reset the index; this step is unnecessary but helpful for viewing the data in chronological checkout order. 

In [2]:
#Read in cleaned data
#df=pd.read_csv('data/Checkouts_2005_cleaned.csv')
#df=pd.read_csv('data/Checkouts_2005_cleaned_w_colon.csv')
df=pd.read_csv('data/Checkouts_2005_no_whitespace.csv')
df = df.dropna()
df['CheckoutDate'] = pd.to_datetime(df['CheckoutDate'])
df = df.sort_values(by='CheckoutDate')
df =df.reset_index(drop=True)

df.head()

,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CleanedCreator,CheckoutDate
0,Physical,BOOK,3,The rosary girls / Richard Montanari.,"Montanari, Richard","Balzano Jessica Fictitious character Fiction, ...","Ballantine Books,",2005,therosarygirls,montanaririchard,2005-04-01
1,Physical,BOOK,2,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv","Money Psychological aspects, Millionaires Psyc...","HarperBusiness,",2005,secretsofthemillionairemind,ekerharvt,2005-04-01
2,Physical,BOOK,7,In the company of liars / David Ellis.,"Ellis, David, 1967-",Thrillers Fiction,"G.P. Putnam's Sons,",2005,inthecompanyofliars,davidellis,2005-04-01
3,Physical,BOOK,21,The motive / John Lescroart.,"Lescroart, John T.","Hardy Dismas Fictitious character Fiction, Soc...","Dutton,",2005,themotive,johnlescroartt,2005-04-01
4,Physical,BOOK,1,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015","Police England London Fiction, Lisson Grove Lo...","Thorndike Press,",2005,therottweiler,rendellruth,2005-04-01


## Aggregating Data: 

Our goal is to make a new dataset where each row corresponds to a unique set of
(CleanedTitle, CleanedCreator, UsageClass, MaterialType). The dataframe will have a column for each month of checkout data we have (April 2005 - April 2024), where these columns contain the number of checkouts in that month for each book respectively. Note that we have data up to August 2024; however, the months of May 2024 and June 2024 are empty. Thus, we take the last month of data as April 2024. 

We first create a list of the months we have checkout data from, where each month is a string of the form MM/DD/YYYY. This list of strings will be used to populate the columns of our dataframe later.

In [3]:
#Get a list of the months we have checkout data from 
months_of_interest=pd.date_range(start='2005-04-01', end='2024-04-01',freq='MS')


In [4]:
print(months_of_interest)

DatetimeIndex(['2005-04-01', '2005-05-01', '2005-06-01', '2005-07-01',
               '2005-08-01', '2005-09-01', '2005-10-01', '2005-11-01',
               '2005-12-01', '2006-01-01',
               ...
               '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01',
               '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01',
               '2024-03-01', '2024-04-01'],
              dtype='datetime64[ns]', length=229, freq='MS')


In [5]:
months_of_interest[-1]


Timestamp('2024-04-01 00:00:00')

In [6]:
#Get the list of all months as strings; will be used as column names later
month_columns = months_of_interest.strftime('%m/%d/%Y')
month_columns = month_columns.to_list()


In [7]:
# Remove rows of df with checkout dates after '04/01/2024'
df = df[df['CheckoutDate'] <= months_of_interest[-1]]

#Verify we have the months we want
for i in df.CheckoutDate.value_counts().index.sort_values():
    print(i)


2005-04-01 00:00:00
2005-05-01 00:00:00
2005-06-01 00:00:00
2005-07-01 00:00:00
2005-08-01 00:00:00
2005-09-01 00:00:00
2005-10-01 00:00:00
2005-11-01 00:00:00
2005-12-01 00:00:00
2006-01-01 00:00:00
2006-02-01 00:00:00
2006-03-01 00:00:00
2006-04-01 00:00:00
2006-05-01 00:00:00
2006-06-01 00:00:00
2006-07-01 00:00:00
2006-08-01 00:00:00
2006-09-01 00:00:00
2006-10-01 00:00:00
2006-11-01 00:00:00
2006-12-01 00:00:00
2007-01-01 00:00:00
2007-02-01 00:00:00
2007-03-01 00:00:00
2007-04-01 00:00:00
2007-05-01 00:00:00
2007-06-01 00:00:00
2007-07-01 00:00:00
2007-08-01 00:00:00
2007-09-01 00:00:00
2007-10-01 00:00:00
2007-11-01 00:00:00
2007-12-01 00:00:00
2008-01-01 00:00:00
2008-02-01 00:00:00
2008-03-01 00:00:00
2008-04-01 00:00:00
2008-05-01 00:00:00
2008-06-01 00:00:00
2008-07-01 00:00:00
2008-08-01 00:00:00
2008-09-01 00:00:00
2008-10-01 00:00:00
2008-11-01 00:00:00
2008-12-01 00:00:00
2009-01-01 00:00:00
2009-02-01 00:00:00
2009-03-01 00:00:00
2009-04-01 00:00:00
2009-05-01 00:00:00


The dataset has a wide variety of Material Types: 

['BOOK' 'SOUNDDISC' 'SOUNDREC' 'SOUNDDISC, VIDEODISC' 'MUSIC' 'AUDIOBOOK'
 'EBOOK' 'VIDEODISC' 'REGPRINT' 'ER' 'VISUAL' 'ATLAS' 'MAP' 'SOUNDCASS'
 'VIDEO' 'ER, MAP' 'ER, SOUNDDISC' 'ER, SOUNDDISC, VIDEODISC'
 'MUSICSNDREC' 'FLASHCARD, SOUNDDISC' 'ER, PRINT' 'ER, NONPROJGRAPH'
 'ER, VIDEODISC' 'LARGEPRINT' 'NOTATEDMUSIC' 'MAP, VIEW'
 'REGPRINT, VIDEOREC' 'KIT' 'REGPRINT, SOUNDDISC' 'ER, REGPRINT'
 'FLASHCARD' 'UNSPECIFIED' 'MIXED' 'BOOK, ER' 'ER, SOUNDREC'
 'ER, SOUNDDISC, SOUNDREC' 'PHOTO']

 To simplify our analysis, we take the five categories with the most checkouts and categorize all other types as 'OTHER'. 

In [8]:
#Valid material types
material_types = ['BOOK','EBOOK','SOUNDDISC','VIDEODISC','AUDIOBOOK', 'OTHER']


In [9]:
#Print original list of MaterialTypes found in dataset
print(df.MaterialType.unique())
#Filter out the material types not in material_types and replace them with "OTHER"
df['MaterialType'] = df['MaterialType'].where(df['MaterialType'].isin(material_types), 'OTHER')
#Print the new list of MaterialTypes found in dataset
print(df.MaterialType.unique())

['BOOK' 'SOUNDDISC' 'SOUNDREC' 'SOUNDDISC, VIDEODISC' 'MUSIC' 'AUDIOBOOK'
 'EBOOK' 'VIDEODISC' 'REGPRINT' 'ER' 'VISUAL' 'ATLAS' 'MAP' 'SOUNDCASS'
 'VIDEO' 'ER, MAP' 'ER, SOUNDDISC' 'ER, SOUNDDISC, VIDEODISC'
 'MUSICSNDREC' 'FLASHCARD, SOUNDDISC' 'ER, PRINT' 'ER, NONPROJGRAPH'
 'ER, VIDEODISC' 'LARGEPRINT' 'NOTATEDMUSIC' 'MAP, VIEW'
 'REGPRINT, VIDEOREC' 'KIT' 'REGPRINT, SOUNDDISC' 'ER, REGPRINT'
 'FLASHCARD' 'UNSPECIFIED' 'MIXED' 'BOOK, ER' 'ER, SOUNDREC'
 'ER, SOUNDDISC, SOUNDREC' 'PHOTO']
['BOOK' 'SOUNDDISC' 'OTHER' 'AUDIOBOOK' 'EBOOK' 'VIDEODISC']


In [87]:
#Sample dataframe used for priliminary testing
df_sample = df.sample(1000)
df_sample = df_sample.sort_values(by='CheckoutDate')
df_sample =df_sample.reset_index(drop=True)
df_sample.head()

,UsageClass,MaterialType,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CleanedTitle,CleanedCreator,CheckoutDate,CheckoutDate_str,index_str
0,Physical,BOOK,10,Why do I love these people? : honest and amazi...,"Bronson, Po, 1964-",Families United States Case studies,"Random House,",2005,whydoilovethesepeople,bronsonpo,2006-02-01,02/01/2006,"whydoilovethesepeople , bronsonpo , Physical ,..."
1,Digital,AUDIOBOOK,1,Tales from Shakespeare (Unabridged),Charles Lamb,"Classic Literature, Fiction, Suspense","Sound Room Publishers, Inc.",2005,talesfromshakespeare,charleslamb,2006-05-01,05/01/2006,"talesfromshakespeare , charleslamb , Digital ,..."
2,Physical,BOOK,3,Washoku : recipes from the Japanese home kitch...,"Andoh, Elizabeth",Cooking Japanese,"Ten Speed Press,",2005,washoku,andohelizabeth,2006-12-01,12/01/2006,"washoku , andohelizabeth , Physical , BOOK"
3,Physical,BOOK,1,Read and learn Bible / [stories retold by Eva ...,"Moore, Eva","Bible Old Testament Juvenile literature, Bible...","Scholastic/American Bible Society,",2005,readandlearnbible,evamoore,2007-02-01,02/01/2007,"readandlearnbible , evamoore , Physical , BOOK"
4,Digital,AUDIOBOOK,1,Cause Celeb (Unabridged),Helen Fielding,"Fiction, Literature","Brilliance Audio, Inc.",2005,causeceleb,fieldinghelen,2007-02-01,02/01/2007,"causeceleb , fieldinghelen , Digital , AUDIOBOOK"


We get the unique sets from (CleanedTitle, CleanedCreator, UsageClass, MaterialType) using value_counts(). We see that we have 621171 (715720 in the case we don't filter out colon parts of title) different possible combinations of these features out of the ~25 million rows in the dataframe. Note that when we only used unique pairs of (CleanedTitle, CleanedCreator), we had around 440,000 different combinations. 

Update: in the event, we remove whitespace from CleanedTitle and CleanedCreator, we obtain 618128 unique sets. 

In [10]:
#Get the unique sets from (CleanedTitle, CleanedCreator, UsageClass, MaterialType)
pairs = df[['CleanedTitle', 'CleanedCreator','UsageClass', 'MaterialType']].value_counts().reset_index()
pairs = pairs.drop(columns='count')
    
pairs.shape

(609183, 4)

We create a numpy array to store the checkout data for each month with rows corresponding to the rows in pairs and columns corresponding to the month in month_columns. We will then convert this to a dataframe to be merged with pairs later, which is significantly faster than updating pairs along the way. 

In [11]:
#Create numpy array of zeros to store the number of checkouts in each month
#Rows: indicate corresponding row in pairs
#Columns: indicate corresponding month in month_columns
months_count = np.zeros((pairs.shape[0], len(month_columns)),dtype = int)

In [13]:
#Get the column names of the features we want to add back in
features = df.columns.to_list()

#Remove the column names we already have in pairs
features.remove('CleanedTitle')
features.remove('CleanedCreator')
features.remove('UsageClass')
features.remove('MaterialType')


print(features)

['Checkouts', 'Title', 'Creator', 'Subjects', 'Publisher', 'PublicationYear', 'CheckoutDate']


The main processing is done in the cell below: 
### Preprocessing: 
To speed up the runtime, we create a few columns with corresponding dictionaries that will enable us to grab row/column indices without searching through the dataframe. We use dictionaries because they are implemented via a hash map so that the average lookup time is O(1), compared with the O(N) we would need if we searched through the dataframe directly. The 'CheckoutDate_str' column converts the 'CheckoutDate' to a string that matches the month strings in month_columns; the dictionary date_to_idx will allow us to go from this string to the appropriate column index of the month we want to update in month_counts. The column 'index_str' contains the unique combination of the (CleanedTitle, CleanedCreator, UsageClass, MaterialType) converted to a single string separated by commas. This simplifies the processing so that we only need to consider one column from the row we have instead of four. We use this column and the indices in pairs to create a dictionary pairs_lookup which will allow us to go from the 'index_str' in df to the corresponding row index in pairs that it corresponds to. This index will be used to update month_counts and the other feature data for each row in pairs. Finally we initialize an array of zeros called filled, which signifies if the additional data corresponding to the column names found in features has been found for each row in pairs. Since df is sorted by CheckoutDate and .apply() is applied top down (first row to last row), we should grab the first CheckoutDate for the book and use its features information to update the row. Note that there is no guarentee that all books of type (CleanedTitle, CleanedCreator, UsageClass, MaterialType) have the same Publisher, Subject, PublicationYear, etc., but they should be similar so we choose to grab the first one. 

### Main Processing: 
updates_months_full is applied to each row of our dataframe df via df.apply(). As it traverses each row, it updates month_counts month index and pairs index it obtains from the date_to_idx and pairs_lookup dictionaries based on the 'CheckoutDate_str' and 'index_str' columns of the row. Additionally, it checks if we have found the additional data corresponding to the column names in features by checking the appropriate index in filled. If not, we return the features values from the row in addition to the row index of pairs. 

### Output: 
After applying updates_months_full to each row in df, the months_count array will contain checkout data from each month for each row in pairs. Additionally, for each row in pairs, updates will contain the corresponding features column information as a list of lists. 

In [14]:
# Convert 'CheckoutDate' to string in the desired format beforehand
df['CheckoutDate_str'] = df['CheckoutDate'].dt.strftime('%m/%d/%Y')

#Create columns for easier lookup in pairs and df; we treat this column as an index column 
pairs['index_str'] = pairs['CleanedTitle'] + " , " + pairs['CleanedCreator'] + " , " \
    + pairs['UsageClass'] + " , " + pairs['MaterialType']

df['index_str'] =  df['CleanedTitle'] + " , " + df['CleanedCreator'] + " , " \
                 + df['UsageClass'] + " , " + df['MaterialType']

#Create dictionary with keys given by index_str and values the corresponding index in pairs
vals = pairs['index_str'].values
indices = pairs.index
pairs_lookup = dict(zip(vals, indices))

#Dictionary to go from CheckoutDate_str to index in list of months
date_to_idx = {date_str: idx for idx, date_str in enumerate(month_columns)}



#Create array indicating which rows of pairs we have already filled in
# 1 indicates we already filled it; 0 indicates we have not
filled= [0] * len(indices)

#Function to aggregate all the needed info
def update_months_full(row):
    #Update months count with checkout information
    if row['CheckoutDate_str'] in date_to_idx:
        date_idx  = date_to_idx[row['CheckoutDate_str']]
    #Only take the months in the month_columns list; else drop row
    else:
        print(row['CheckoutDate_str'])
        return None
    pairs_idx = pairs_lookup[row['index_str']]
    months_count[pairs_idx, date_idx] +=  row['Checkouts']

    #Update additional information 
    title_creator_str = row['index_str']
    #Check if we have updated the row yet; if not, update it
    if filled[pairs_lookup[title_creator_str]]==0:
        #Change indicator to 1 to indicate we have visited row
        filled[pairs_lookup[title_creator_str]]=1
        #Return the index in pairs with the corresponding features from df
        return list(np.insert(row[features].values, 0, pairs_lookup[title_creator_str]))
    return None



# Apply the update_row function to each row in df
updates = df.apply(update_months_full, axis=1)
updates = updates.dropna()
assert(updates.shape[0]==pairs.shape[0])
updates.head()



0    [161204, 3, The rosary girls / Richard Montana...
1    [2697, 2, Secrets of the millionaire mind : ma...
2    [61033, 7, In the company of liars / David Ell...
3    [23674, 21, The motive / John Lescroart., Lesc...
4    [146945, 1, The rottweiler [text (large print)...
dtype: object

In [15]:
print(updates.shape[0])
print(pairs.shape[0])

609183
609183


In [16]:
#Make sure pairs and updates have the same number of rows
print(pairs.shape)
print(updates.shape)


(609183, 5)
(609183,)


Now that we have all the information we need, we will create dataframes for updates and month_counts and merge them with pairs. We create dataframes via pd.DataFrame on month_counts and updates. updates already contains an index column which corresponds to the indices in pairs. To merge on these indices, we create corresponding 'index' columns in pairs and months_df via reset_index(). Note that the indicies of months_df already match those of pairs due to the construction of month_counts.  Finally, we merge updates_df with pairs and then the result of that with month_df on the index column to obtain the final dataframe merged_df. We drop some of the extra columns we created for processing and finally write the data to the file cleaned_months_with_types.csv in the data folder. 

In [17]:
# Features we obtained in updates 
# We added a column for the corresponding index of pairs, so we add the column name here
features_new = ['index', *features]
print(features_new)

['index', 'Checkouts', 'Title', 'Creator', 'Subjects', 'Publisher', 'PublicationYear', 'CheckoutDate']


In [18]:
#Remove the extra column we made for indexing convenience
pairs = pairs.drop(columns=['index_str'])

#Add column with index of each row in pairs for merging 
pairs = pairs.reset_index()
pairs.head()

,index,CleanedTitle,CleanedCreator,UsageClass,MaterialType
0,0,fancynancy,connorjaneo,Physical,BOOK
1,1,yugioh,kazukitakahashi,Physical,BOOK
2,2,thewalkingdead,kirkmanrobert,Physical,BOOK
3,3,nationalgeographicreaders,lauramarsh,Digital,EBOOK
4,4,avatar,geneluenyang,Digital,EBOOK


In [19]:
#Create updates dataframe to be joined with pairs
#The index column corresponds to the index column of pairs
updates_df =pd.DataFrame(updates.to_list(), columns=features_new)
updates_df.head()



,index,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate
0,161204,3,The rosary girls / Richard Montanari.,"Montanari, Richard","Balzano Jessica Fictitious character Fiction, ...","Ballantine Books,",2005,2005-04-01
1,2697,2,Secrets of the millionaire mind : mastering th...,"Eker, T. Harv","Money Psychological aspects, Millionaires Psyc...","HarperBusiness,",2005,2005-04-01
2,61033,7,In the company of liars / David Ellis.,"Ellis, David, 1967-",Thrillers Fiction,"G.P. Putnam's Sons,",2005,2005-04-01
3,23674,21,The motive / John Lescroart.,"Lescroart, John T.","Hardy Dismas Fictitious character Fiction, Soc...","Dutton,",2005,2005-04-01
4,146945,1,The rottweiler [text (large print)] / Ruth Ren...,"Rendell, Ruth, 1930-2015","Police England London Fiction, Lisson Grove Lo...","Thorndike Press,",2005,2005-04-01


In [20]:
#Create months dataframe containing the counts for each month; to be joined with pairs
# Note arrays index with (0,0) as top left so the index column is aligned with the index
# columns of pairs
months_df = pd.DataFrame(months_count, columns=month_columns)
#Add index column to match with pairs
months_df = months_df.reset_index()
months_df.head()


,index,04/01/2005,05/01/2005,06/01/2005,07/01/2005,08/01/2005,09/01/2005,10/01/2005,11/01/2005,12/01/2005,...,07/01/2023,08/01/2023,09/01/2023,10/01/2023,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024
0,0,0,0,0,0,0,0,0,0,0,...,86,90,80,86,82,52,71,69,54,68
1,1,0,0,0,0,0,0,0,0,0,...,4,15,4,7,5,3,1,2,1,1
2,2,0,0,0,0,0,0,0,0,0,...,4,4,6,7,6,6,7,14,15,1
3,3,0,0,0,0,0,0,0,0,0,...,2,5,2,6,2,3,4,4,2,5
4,4,0,0,0,0,0,0,0,0,0,...,210,151,176,169,137,166,190,246,382,348


In [21]:
merged1_df = pd.merge(updates_df, pairs, on='index')
merged_df = pd.merge(merged1_df, months_df, on='index')

#Sort by index so that when 
merged_df = merged_df.sort_values(by='index')
#Remove the index and Checkouts columns as they are unnecessary. 
merged_df = merged_df.drop(columns=['index', 'Checkouts'])
merged_df.head()


,Title,Creator,Subjects,Publisher,PublicationYear,CheckoutDate,CleanedTitle,CleanedCreator,UsageClass,MaterialType,...,07/01/2023,08/01/2023,09/01/2023,10/01/2023,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024
8266,Fancy Nancy / by Jane O'Connor ; pictures by R...,"O'Connor, Jane",Fancy Nancy Fictitious character Juvenile fict...,"HarperCollins,",2006,2006-03-01,fancynancy,connorjaneo,Physical,BOOK,...,86,90,80,86,82,52,71,69,54,68
12978,"Yu-Gi-Oh! : duelist. Vol. 12, Magician vs. mag...","Takahashi, Kazuki, 1961-",Games Comic books strips etc Juvenile literatu...,"Viz Media,",2005,2006-08-01,yugioh,kazukitakahashi,Physical,BOOK,...,4,15,4,7,5,3,1,2,1,1
45510,"The walking dead . [Volume 2, Miles behind us]...","Kirkman, Robert","Zombies Comic books strips etc, Survivalism Co...","Image Comics,",2006,2008-10-01,thewalkingdead,kirkmanrobert,Physical,BOOK,...,4,4,6,7,6,6,7,14,15,1
285605,National Geographic Readers: Lizards,Laura Marsh,"Beginning Reader, Juvenile Nonfiction, Nature","Random House, Inc.",2015,2015-07-01,nationalgeographicreaders,lauramarsh,Digital,EBOOK,...,2,5,2,6,2,3,4,4,2,5
296436,"Avatar: The Last Airbender - Smoke and Shadow,...",Gene Luen Yang,"Comic and Graphic Books, Juvenile Fiction, Juv...","Random House, Inc.",2015,2015-10-01,avatar,geneluenyang,Digital,EBOOK,...,210,151,176,169,137,166,190,246,382,348


In [22]:
#Make sure data matches 
merged_df[['Title', 'Creator', 'CleanedTitle', 'CleanedCreator']].head(50)

,Title,Creator,CleanedTitle,CleanedCreator
8266,Fancy Nancy / by Jane O'Connor ; pictures by R...,"O'Connor, Jane",fancynancy,connorjaneo
12978,"Yu-Gi-Oh! : duelist. Vol. 12, Magician vs. mag...","Takahashi, Kazuki, 1961-",yugioh,kazukitakahashi
45510,"The walking dead . [Volume 2, Miles behind us]...","Kirkman, Robert",thewalkingdead,kirkmanrobert
285605,National Geographic Readers: Lizards,Laura Marsh,nationalgeographicreaders,lauramarsh
296436,"Avatar: The Last Airbender - Smoke and Shadow,...",Gene Luen Yang,avatar,geneluenyang
214882,The Year's Best Science Fiction: Fifth Annual ...,Gardner Dozois,theyearsbestsciencefiction,dozoisgardner
12113,Ranma 1/2 Vol. 34 / story & art by Rumiko Taka...,"Takahashi, Rumiko, 1957-",ranma1,rumikotakahashi
200986,Fly Guy presents : sharks / Tedd Arnold.,"Arnold, Tedd",flyguypresents,arnoldtedd
220080,Magi : the labyrinth of magic. 1 / story & art...,"Ōtaka, Shinobu",magi,shinobuōtaka
175438,Invincible : ultimate collection. Volume 2 / c...,"Kirkman, Robert",invincible,kirkmanrobert


In [23]:
merged_df.shape

(609183, 239)

In [24]:
#merged_df.to_csv('data/cleaned_months_with_type_2005.csv', index=False)
merged_df.to_csv('data/cleaned_months_no_whitespace_2005.csv', index=False)

## Some additional exploration: 

We note that of the materials we have, most of them are physical books, followed by ebooks, audiobooks, sounddiscs. A small proportion is composed of other with both physical and digital classes and finally there are few videodiscs.

In [25]:
pairs = merged_df[['UsageClass', 'MaterialType']].value_counts().reset_index()
pairs.head(pairs.shape[0])

,UsageClass,MaterialType,count
0,Physical,BOOK,295250
1,Digital,EBOOK,186458
2,Digital,AUDIOBOOK,73360
3,Physical,SOUNDDISC,44315
4,Digital,OTHER,5186
5,Physical,OTHER,4244
6,Physical,VIDEODISC,370


We verify that the date in CheckoutDate does contain the first checkout date. It should since we sorted our dataframe by CheckoutDate before processing, and here we verify this: 

In [28]:
#Format CheckoutDate column to match column name
merged_df['CheckoutDate_str'] = merged_df['CheckoutDate'].dt.strftime('%m/%d/%Y')

#Get indices of first and last months
first_index= merged_df.columns.get_loc('04/01/2005')
last_index =  merged_df.columns.get_loc('04/01/2024')

#Create new dataframe with just month data to process
checkouts = merged_df.iloc[:,first_index:last_index+1]

#Function to grab the first nonzero column in each row of checkouts
def find_first_nonzero_column(row):
    return row[row.ne(0)].index[0]

#Apply function to each row in df_sample
merged_df['first_nonzero_month'] = checkouts.apply(find_first_nonzero_column, axis=1)

#Check that the values match in each column
checking = merged_df['first_nonzero_month']== merged_df['CheckoutDate']
#This prints True if all match; false otherwise
print(any(checking))

#Drop the columns added
merged_df = merged_df.drop(columns=['CheckoutDate_str', 'first_nonzero_month'])


True


## Additional Post-Processing

We would like to clean the Subject column and assign specific genres to each book. We do so via the classify_genre, in which we lower the Subject column and find matches with our predefined genre list. Note that since we want just one genre per book, the function prioritizes genres in the order they are listed with very specific things first and more generic genres like "fiction" and "nonfiction" last.

In [78]:
#Read in data again or rename as df for convenience
#df = merged_df
#df = pd.read_csv('data/cleaned_months_with_type_2005.csv')
df = pd.read_csv('data/cleaned_months_no_whitespace_2005.csv')
df['CheckoutDate'] = pd.to_datetime(df['CheckoutDate'])

In [79]:
df.shape

(609183, 239)

In [81]:
df['CleanedSubject'] = df['Subjects'].str.lower().str.strip()

# Define the list of genres in the desired hierarchy order
genres = ['juvenile', 'young adult', 'fantasy', 'romance', 'thriller',\
            'horror', 'mystery', 'science fiction', 'biography', \
                'history', 'novel', 'nonfiction', 'fiction']
# Make list of true genre names we want to consider for genre in genres
# This list aligns with the placement in genres
# For example, fantasy becomes fantasy/sci-fi as does science fiction
genres_match = ['juvenile', 'young adult', 'fantasy/sci-fi', 'romance', 'horror/thriller',\
            'horror/thriller', 'mystery', 'fantasy/sci-fi', 'biography', \
                'history', 'fiction', 'nonfiction', 'fiction']
#Make another list of the desired final genre list without duplicates
genres_final = ['juvenile', 'young adult', 'fantasy/sci-fi', 'romance', \
                'horror/thriller', 'mystery', 'biography', 'history', \
                 'nonfiction', 'fiction']

#Create dictionary combining genres found in Subject string
#  with genre names we want to use
genres_final_dict = dict(zip(genres, genres_match))

# Create a dictionary for priority of each genre
priority = {genre: i for i, genre in enumerate(genres_final)}

# Function to classify genre based on the CleanedSubject column
def classify_genre(cleaned_subject):
    if pd.isna(cleaned_subject):  # If the entry is NaN
        return 'other'

    # Convert the cleaned_subject to lower case for case-insensitive matching
    cleaned_subject = cleaned_subject.lower()

    # Create a dictionary for priority of each genre
    #priority = {genre: i for i, genre in enumerate(genres)} 

    # Find all matching genres in the cleaned_subject string
    #found_genres = [genre for genre in genres if genre in cleaned_subject]
    found_genres = [genres_final_dict[genre] for genre in genres if genre in cleaned_subject]


    if not found_genres:  # If no genre is found
        return 'other'
    

    # Sort genres based on their priority
    found_genres.sort(key=lambda g: priority[g])

    # Return the genre with the highest priority
    return found_genres[0]

# Apply the function to the dataframe
df['Genre'] = df['CleanedSubject'].apply(classify_genre)

We consider the distribution of the genres, and see that out of the ~610000 possibilities, we have around 122000 'other' options. Thus, most of the books can be characterized into one of the possible genres. 

In [82]:
genre_counts = df['Genre'].value_counts().reset_index()

genre_counts.head(20)

,Genre,count
0,juvenile,123928
1,other,122386
2,fiction,82675
3,nonfiction,53144
4,biography,42914
5,fantasy/sci-fi,37867
6,mystery,35584
7,history,32934
8,romance,31169
9,horror/thriller,26350


We now want to aggregate the number of checkouts in the first year, with the first checkout date denoting the publication date. We do so via the sum_checkouts_first_year function defined below, which is applied to each row in our dataframe. This function grabs the first checkoutdate of each item, located in the CheckoutDate column, and calculates the date that is a year out. From the columns_dict dictionary, which has keys the column names of df and values the corresponding indices, we can them grab the appropriate columns indices to sum over to get the total number of checkouts in the first year.  Note that if the first checkout date is after 09/01/2023, we only sum the partial year up to 08/01/2024. We store the result into a new column "FirstYearCheckout"

In [83]:
#Create dictionary of column, index pairs to 
indices = list(range(len(df.columns)))
columns = df.columns
columns_dict = dict(zip(columns, indices))


#Function to sum checkouts from first checkout date to one year later
def sum_checkouts_first_year(row):
    first_checkout = row['CheckoutDate'] #Get first checkout date
    first_index= columns_dict[first_checkout.strftime('%m/%d/%Y')] #Convert to column name form and get index of column
    end_year  = (first_checkout.replace(year=first_checkout.year +1)).strftime('%m/%d/%Y') #Add one to year to get end date
    if end_year in columns_dict: #Get index of end year column
        end_index= columns_dict[end_year]
    else:
        #Get last possible month of data and include it; which gives the +1
        end_index =columns_dict['04/01/2024'] +1 
    checkout_sum = 0
    for i in range(first_index, end_index): #Sum over all checkouts in range
        checkout_sum += row.iloc[i]
    return checkout_sum

df['FirstYearCheckouts'] = df.apply(sum_checkouts_first_year, axis = 1)
    

Our next task it get the sum of checkouts per month per author in the year prior to the first checkout. Note that if the start of the previous year is outside the scope of the data we have, we do a partial sum of whatever months we are able; e.g. if the checkout date was 01/01/2006, then we would sum the authors checkouts from 04/01/2005 to 12/01/2005 since our first checkout is April 2005. To create the previous checkout column, we first get the sum of all book checkouts per month per author, which is stored in author_summed. Then for each row in our dataframe, we get the first checkout date and sum the appriopriate columns in author_summed to get total checkouts in the previous year for author corresponding to that row. We store the result into a column named PreviousYearCheckouts. 

In [84]:
#Get index range for columns of interest
first_index= columns_dict['04/01/2005']
last_index = columns_dict['04/01/2024']

#Get dataframe with CleanedCreator column plus all corresponding months column
months_df = df.iloc[:, first_index:last_index+1 ]
months_df.insert(loc = 0,column='CleanedCreator', value=df.CleanedCreator)

#Get dataframe with CleanedCreator column plus all corresponding months column
months_df = df.iloc[:, first_index:last_index+1 ]
months_df.insert(loc = 0,column='CleanedCreator', value=df.CleanedCreator)

#Sum the checkouts for each author in each month
# For each author, we have all months columns filled with all checkouts in that month
author_summed = months_df.groupby('CleanedCreator').sum()

author_summed = author_summed.reset_index()

author_summed.head()


,CleanedCreator,04/01/2005,05/01/2005,06/01/2005,07/01/2005,08/01/2005,09/01/2005,10/01/2005,11/01/2005,12/01/2005,...,07/01/2023,08/01/2023,09/01/2023,10/01/2023,11/01/2023,12/01/2023,01/01/2024,02/01/2024,03/01/2024,04/01/2024
0,aa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,aaaguirre,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,1,3,0
2,aaakerdavid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,aaanuel,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,aabalaskovits,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,1,0


In [85]:
#Create dictionary of column, index pairs for author_summed
indices = list(range(len(author_summed.columns)))
columns = author_summed.columns
author_columns_dict = dict(zip(columns, indices))

#Create dictionary to get row of author_summed by CleanedCreator 
author_dict = dict(zip(author_summed.CleanedCreator, author_summed.index))

# Function to calcuate number of checkouts of the author in the year before first checkout
def previous_year_sum(row):
    first_checkout = row['CheckoutDate']
    author = row['CleanedCreator']
    idx = author_dict[author]
    end_index= author_columns_dict[first_checkout.strftime('%m/%d/%Y')] #Convert to column name form and get index of column
    previous_year  = (first_checkout.replace(year=first_checkout.year -1)).strftime('%m/%d/%Y') #Add one to year to get end date
    if previous_year in author_columns_dict: #Get index of end year column
        first_index= author_columns_dict[previous_year]
    else:
        # Get first possible month of data
        first_index =author_columns_dict['04/01/2005']
    checkout_sum = 0
    for i in range(first_index, end_index): #Sum over all checkouts in range
        checkout_sum += author_summed.iloc[idx, i]
    return checkout_sum


df['PreviousYearCheckouts'] = df.apply(previous_year_sum, axis=1)
    

With our clean data, we want to create a train test split. We create and drop some columns to make processing easier later and then make a train test split. Note that we will have books with checkout dates before April 2006 in both training and testing data sets, which will need care in the future. 

In [98]:
# Create columns for month and year to be used as possible features later
df['CheckoutMonth'] = df['CheckoutDate'].dt.month
df['CheckoutYear'] = df['CheckoutDate'].dt.year

In [107]:
#Remove the columns of all the months of the form MM/DD/YYYY
indices = list(range(len(df.columns)))
columns = df.columns
columns_dict = dict(zip(columns, indices))

#Find the indices of the first and last columns
first_index= columns_dict['04/01/2005']
last_index = columns_dict['04/01/2024']

dropped_months = columns[first_index:last_index+1].to_list()

dropped_columns = [*dropped_months, 'Subjects', 'CleanedSubject',\
                   'CleanedTitle', 'CleanedCreator', 'PublicationYear', 'CheckoutDate']

df = df.drop(columns=dropped_columns)

In [112]:
df.head()

,Title,Creator,Publisher,UsageClass,MaterialType,Genre,FirstYearCheckouts,PreviousYearCheckouts,CheckoutMonth,CheckoutYear
0,Fancy Nancy / by Jane O'Connor ; pictures by R...,"O'Connor, Jane","HarperCollins,",Physical,BOOK,juvenile,131,0,3,2006
1,"Yu-Gi-Oh! : duelist. Vol. 12, Magician vs. mag...","Takahashi, Kazuki, 1961-","Viz Media,",Physical,BOOK,juvenile,604,0,8,2006
2,"The walking dead . [Volume 2, Miles behind us]...","Kirkman, Robert","Image Comics,",Physical,BOOK,horror/thriller,169,772,10,2008
3,National Geographic Readers: Lizards,Laura Marsh,"Random House, Inc.",Digital,EBOOK,juvenile,186,504,7,2015
4,"Avatar: The Last Airbender - Smoke and Shadow,...",Gene Luen Yang,"Random House, Inc.",Digital,EBOOK,juvenile,217,2997,10,2015


In [108]:
# Split data into 80% training data and 20% testing data
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2,\
                                      shuffle=True, random_state=216)

In [111]:
df_train.to_csv('data/cleaned_train_2005.csv', index=False)
df_test.to_csv('data/cleaned_test_2005.csv', index=False)

In [114]:
print(df_train.shape)
print(df_test.shape)

(487346, 10)
(121837, 10)
